<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Regridding TERRACLIMATE with xesmf

_by Joe Hamman (CarbonPlan), June 29, 2020_

This notebook converts the raw TERAACLIMATE dataset to Zarr format.

**Inputs:**

**Outputs:**

- Cloud copy of TERRACLIMATE

**Notes:**

- No reprojection or processing of the data is done in this notebook.


In [ ]:
pip install -U xarray==0.16.0 --no-deps

In [ ]:
import fsspec
import xarray as xr
import xesmf as xe
import numpy as np

from dask.diagnostics import ProgressBar

In [ ]:
variables = {
    #     'conservative': [
    #         "aet",
    #         "def",
    #         "pet",
    #         "ppt",
    #         "q",
    #         "srad",
    #     ],
    "bilinear": [
        "tmax",
        "tmin",
        "pdsi",
        "vap",
        "vpd",
        "ws",
        "soil",
        "swe",
        # move to conservative after scrable is fixed
        "aet",
        "def",
        "pet",
        "ppt",
        "q",
        "srad",
        "awc",
        "elevation",
    ]
}

In [ ]:
# options
name = "terraclimate"
raw_location = f"gs://carbonplan-data/raw/terraclimate/4000m/raster.zarr"
target_grid = "gs://carbonplan-data/processed/grids/conus/4000m/domain.zarr"
# getting weird errors when writing to carbonplan-data
target_location = (
    f"gs://carbonplan-data/processed/{name}/conus/4000m/raster.zarr"
)

In [ ]:
mapper = fsspec.get_mapper(target_grid)
target_ds = xr.open_zarr(
    mapper, consolidated=True
)  # .rename({'xc': 'lon', 'yc': 'lat'})
target_ds

In [ ]:
mapper = fsspec.get_mapper(raw_location)
ds = xr.open_zarr(mapper, consolidated=True)
ds

In [ ]:
step = 360 / 8640 + 1e-9
global_grid = xe.util.grid_global(step, step)
global_grid = global_grid.isel(y=slice(None, None, -1)).isel(
    y_b=slice(None, None, -1)
)
global_grid["lat_b"].values = np.clip(global_grid["lat_b"].values, -90, 90)
display(global_grid)

# check that this grid is a drop in replacement for the source grid
assert np.abs(global_grid.lat.isel(x=0).values - ds.lat.values).max() < 1e-5
assert np.abs(global_grid.lon.isel(y=0).values - ds.lon.values).max() < 1e-5
assert np.abs(global_grid.lat).max().item() <= 90
assert np.abs(global_grid.lat_b).max().item() <= 90

# rename grid variables
source_ds = ds.rename({"lon": "x", "lat": "y"}).assign_coords(
    coords=global_grid.coords
)

In [ ]:
regridders = {}

for method in variables:
    regridders[method] = xe.Regridder(
        source_ds, target_ds, method, reuse_weights=True
    )

In [ ]:
temp = []
for method, var_list in variables.items():
    regridder = regridders[method]
    temp.append(regridder(ds[var_list].chunk({"lat": -1, "lon": -1})))

ds_out = xr.merge(temp, compat="override")

In [ ]:
ds_out

In [ ]:
# fs = fsspec.get_filesystem_class('gs')()
# fs.rm(target_location, recursive=True)

In [ ]:
import dask
from multiprocessing.pool import ThreadPool

with dask.config.set(scheduler="threads", pool=ThreadPool(3)):
    with ProgressBar():
        mapper2 = fsspec.get_mapper(target_location)
        ds_out.to_zarr(mapper2, mode="w", consolidated=True)

In [ ]:
mapper2 = fsspec.get_mapper(target_location)
import zarr

zarr.consolidate_metadata(mapper2)